In [8]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [9]:
# Input Data Set #
Data ="MONK3"
JobNameAbbrev = "M3"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.011],
                       "Type": ["Classification"]}

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Random Forest Simulations

In [10]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.00],
                       "rashomon_bound_adder": [0],
                       "Type": ["Classification"]}
RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [11]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["rashomon_bound_adder"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_DPL", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_UNQ", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True))

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [12]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,MONK3,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.011,Classification,0M3_DPL0.011,MONK3/TreeFarms/Raw/0M3_DPL0.011.pkl
1,MONK3,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.011,Classification,0M3_UNQ0.011,MONK3/TreeFarms/Raw/0M3_UNQ0.011.pkl
2,MONK3,0,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.00,0.000,Classification,0M3_RF0.0,MONK3/RandomForestClassification/Raw/0M3_RF0.0...
3,MONK3,1,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.00,0.000,Classification,1M3_RF0.0,MONK3/RandomForestClassification/Raw/1M3_RF0.0...
4,MONK3,1,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.011,Classification,1M3_UNQ0.011,MONK3/TreeFarms/Raw/1M3_UNQ0.011.pkl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,MONK3,98,0.2,0.8,TreeEnsembleQBCFunction,RandomForestClassificationFunction,0,100,0.00,0.000,Classification,98M3_RF0.0,MONK3/RandomForestClassification/Raw/98M3_RF0....
296,MONK3,98,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.011,Classification,98M3_UNQ0.011,MONK3/TreeFarms/Raw/98M3_UNQ0.011.pkl
297,MONK3,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.011,Classification,99M3_DPL0.011,MONK3/TreeFarms/Raw/99M3_DPL0.011.pkl
298,MONK3,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.011,Classification,99M3_UNQ0.011,MONK3/TreeFarms/Raw/99M3_UNQ0.011.pkl


In [13]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

---

In [14]:
# ### Simulations that failed ###
# FilterText = ['66IS_UNQ0.041', 
# '66IS_UNQ0.043', 
# '66IS_UNQ0.045', 
# '13IS_UNQ0.045', 
# '66IS_DPL0.041', 
# '66IS_DPL0.043', 
# '66IS_DPL0.045', 
# '13IS_DPL0.045']

# ### Rerun these simulations ###
# ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector
# ### Resave ###
# output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
# ParameterVector.to_csv(output_path, index=False)